In [2]:
import time
import torch
import numpy as np
from vmas import make_env
from vmas.simulator.core import Agent
from vmas.simulator.scenario import BaseScenario
from typing import Union
from moviepy.editor import ImageSequenceClip
from IPython.display import HTML, display as ipython_display

class VMASEnvRunner:
    def __init__(
        self,
        render: bool,
        num_envs: int,
        n_steps: int,
        device: str,
        scenario: Union[str, BaseScenario],
        continuous_actions: bool,
        random_action: bool,
        **kwargs
    ):
        self.render = render
        self.num_envs = num_envs
        self.n_steps = n_steps
        self.device = device
        self.scenario = scenario
        self.continuous_actions = continuous_actions
        self.random_action = random_action
        self.kwargs = kwargs
        self.frame_list = []  # Initialize frame list
        self.q_tables = {}  # Dictionary to store Q-tables for each agent

    def _update_q_table(self, agent_index, state, action, reward, next_state, learning_rate, discount_factor):
        # Implement Q-learning update rule for the specific agent
        if agent_index not in self.q_tables:
            self.q_tables[agent_index] = {}  # Initialize Q-table for the agent
        if state not in self.q_tables[agent_index]:
            self.q_tables[agent_index][state] = np.zeros((num_actions,))  # Initialize Q-values for new state
        if next_state not in self.q_tables[agent_index]:
            self.q_tables[agent_index][next_state] = np.zeros((num_actions,))  # Initialize Q-values for new state

        self.q_tables[agent_index][state][action] += learning_rate * (reward + discount_factor * np.max(self.q_tables[agent_index][next_state]) - self.q_tables[agent_index][state][action])

    def _get_deterministic_action(self, agent, env, agent_index, state):
        if not self.random_action:
            if agent_index not in self.q_tables:
                self.q_tables[agent_index] = {}  # Initialize Q-table for the agent
            if state not in self.q_tables[agent_index]:
                self.q_tables[agent_index][state] = np.zeros((num_actions,))  # Initialize Q-values for new state
            action = np.argmax(self.q_tables[agent_index][state])  # Choose action with highest Q-value
        else:
            action = env.get_random_action(agent)
        return torch.tensor([action], device=env.device, dtype=torch.long).unsqueeze(-1).expand(env.batch_dim, 1)

    def generate_gif(self, scenario_name):
        fps = 30
        clip = ImageSequenceClip(self.frame_list, fps=fps)
        clip.write_gif(f'{scenario_name}.gif', fps=fps)

        # Return the HTML tag to display the GIF
        return HTML(f'<img src="{scenario_name}.gif">')

    def run_vmas_env(self, num_actions, learning_rate, discount_factor):
        scenario_name = self.scenario if isinstance(self.scenario, str) else self.scenario.__class__.__name__

        env = make_env(
            scenario=self.scenario,
            num_envs=self.num_envs,
            device=self.device,
            continuous_actions=self.continuous_actions,
            seed=0,
            **self.kwargs
        )

        init_time = time.time()
        step = 0

        for s in range(self.n_steps):
            step += 1
            print(f"Step {step}")

            actions = []
            for i, agent in enumerate(env.agents):
                state = env.reset()  # Get current state
                if not self.random_action:
                    action = self._get_deterministic_action(agent, env, i, state)
                else:
                    action = env.get_random_action(agent)
                
                next_state, reward, dones, info = env.step(actions)
                self._update_q_table(i, state, action.item(), reward, next_state, learning_rate, discount_factor)
                actions.append(action)


            if self.render:
                frame = env.render(
                    mode="rgb_array",
                    agent_index_focus=None,
                )
                self.frame_list.append(frame)  # Append frame to the list

        total_time = time.time() - init_time
        print(
            f"It took: {total_time}s for {self.n_steps} steps of {self.num_envs} parallel environments on device {self.device} "
            f"for {scenario_name} scenario."
        )

if __name__ == "__main__":
    scenario_name = "waterfall"

    num_actions = 3  # Number of actions
    learning_rate = 0.1  # Learning rate
    discount_factor = 0.9  # Discount factor

    env_runner = VMASEnvRunner(
        render=True,
        num_envs=32,
        n_steps=10,
        device="cuda",
        scenario=scenario_name,
        continuous_actions=False,
        random_action=False,
        # Environment specific variables
        n_agents=2,
    )
    # Run the VMAS environment
    env_runner.run_vmas_env(num_actions, learning_rate, discount_factor)

    # Generate and display the GIF
    ipython_display(env_runner.generate_gif(scenario_name))


Step 1


TypeError: unhashable type: 'list'